
# Pipeline de Coleta → Extração → Limpeza → Deduplicação → Shards (Notebook)

Este notebook executa o pipeline em **etapas**, permitindo inspeção e depuração fase a fase.

> **Tema**: violência doméstica no contexto da atuação policial  
> **Entrada**: `crawl_manifest.jsonl` (manifesto deduplicado com URLs, **formato JSONL**)  
> **Saída**: diretório `OUTDIR/` com brutos (`data/raw`), textos limpos (`data/text`), shards JSONL (`data/shards`) e relatórios (`logs/`)


## 1) Configurações

In [ ]:
#!pip install jsonlines
#!pip install tabulate
!pip install PyMuPDF pdfminer.six

In [1]:

from pathlib import Path
import os

# Caminhos
MANIFEST = Path("ds_grupos_vulneraveis.jsonl")  # <--- ALTERADO: JSONL
OUTDIR = Path("corpus_out_nb")

# Parâmetros
RATE = 1.0                # requisições por segundo (global)
MAX_WORKERS = 6           # threads de download/extração
TIMEOUT = 15              # timeout por requisição (s)
MIN_CHARS = 300           # mínimo de caracteres por doc (após limpeza)
SHARD_SIZE_MB = 100.0     # tamanho alvo de cada shard
LOG_LEVEL = "INFO"        # "DEBUG" para mais verbosidade
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119 Safari/537.36"

# Preparar diretórios de saída
for sub in ["data/raw", "data/text", "data/shards", "logs"]:
    (OUTDIR / sub).mkdir(parents=True, exist_ok=True)

print("OUTDIR:", OUTDIR.resolve())


OUTDIR: /home/ricardo/Documentos/PESSOAL/Mestrado/Dataset/corpus_out_nb


## 2) Imports e utilitários

In [2]:

import csv, re, json, time, hashlib, logging
from urllib.parse import urlparse, urlunparse
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
from bs4 import BeautifulSoup

# Adicionar importação de jsonlines
try:
    import jsonlines
except ImportError:
    logging.error("A biblioteca 'jsonlines' é necessária. Instale com 'pip install jsonlines'")
    jsonlines = None

# Dependências opcionais (tratadas com fallback)
try:
    import trafilatura
except Exception:
    trafilatura = None

try:
    import fitz  # PyMuPDF
except Exception:
    fitz = None

try:
    from pdfminer.high_level import extract_text as pdfminer_extract_text
except Exception:
    pdfminer_extract_text = None

# Logging
logging.basicConfig(level=getattr(logging, LOG_LEVEL.upper(), logging.INFO),
                    format="%(asctime)s %(levelname)s: %(message)s")
log = logging.getLogger("pipeline_nb")

# Sessão HTTP com retries e headers de navegador
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def build_session(timeout: int, user_agent: str):
    s = requests.Session()
    retries = Retry(total=3, backoff_factor=0.8, status_forcelist=[429,500,502,503,504])
    s.mount("http://", HTTPAdapter(max_retries=retries))
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.headers.update({
        "User-Agent": user_agent,
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "pt-BR,pt;q=0.9,en;q=0.8",
        "Connection": "keep-alive",
    })
    orig = s.request
    def wrapped(method, url, **kwargs):
        kwargs.setdefault("timeout", timeout)
        return orig(method, url, **kwargs)
    s.request = wrapped
    return s

session = build_session(TIMEOUT, USER_AGENT)

def sha1(s: str) -> str: return hashlib.sha1(s.encode("utf-8")).hexdigest()
def sha256(s: str) -> str: return hashlib.sha256(s.encode("utf-8")).hexdigest()

def norm_url(u: str) -> str:
    p = urlparse(u); path = p.path.rstrip("/")
    return urlunparse((p.scheme, p.netloc.lower(), path, "", "", ""))

# Rate limiter simples (token bucket)
class RateLimiter:
    def __init__(self, rate_per_sec: float):
        import threading
        self.rate = max(rate_per_sec, 0.1)
        self.tokens = self.rate
        self.last = time.monotonic()
        self.lock = threading.Lock()
    def acquire(self):
        with self.lock:
            while True:
                now = time.monotonic()
                elapsed = now - self.last
                self.tokens = min(self.rate, self.tokens + elapsed * self.rate)
                self.last = now
                if self.tokens >= 1.0:
                    self.tokens -= 1.0
                    return
                time.sleep((1.0 - self.tokens) / self.rate)

limiter = RateLimiter(RATE)

print("Setup complete.")


Setup complete.


## 3) Checagem de `robots.txt` (com timeout)

In [3]:

from urllib import robotparser

class RobotsCache:
    """Busca robots.txt com requests e timeout; se falhar, assume permitido."""
    def __init__(self, user_agent: str, timeout: int = 15):
        self.user_agent = user_agent
        self.timeout = timeout
        self.cache = {}
    def can_fetch(self, url: str) -> bool:
        p = urlparse(url)
        base = f"{p.scheme}://{p.netloc}"
        if base in self.cache:
            rp = self.cache[base]
            if rp is None: return True
            try:
                return rp.can_fetch(self.user_agent, url)
            except Exception:
                return True
        robots_url = base + "/robots.txt"
        rp = robotparser.RobotFileParser()
        try:
            resp = session.get(robots_url)  # session já tem timeout
            if resp.status_code != 200 or not resp.content:
                self.cache[base] = None
                return True
            rp.parse(resp.text.splitlines())
            self.cache[base] = rp
            return rp.can_fetch(self.user_agent, url)
        except Exception:
            self.cache[base] = None
            return True

robots = RobotsCache(USER_AGENT, timeout=TIMEOUT)

print("Setup complete.")

Setup complete.


## 4) Carregar manifesto e deduplicar URLs (Lendo JSONL)

In [3]:

import pandas as pd

if not jsonlines:
    raise ImportError("A biblioteca 'jsonlines' não foi importada corretamente. Verifique a instalação.")

records = []
try:
    with open(MANIFEST, 'r', encoding='utf-8') as f:
        reader = jsonlines.Reader(f)
        for obj in reader:
            # Garantir que a URL e o título existam para validade básica
            if 'url' in obj and 'title' in obj:
                records.append(obj)
            else:
                log.warning(f"Registro ignorado no manifesto JSONL (Falta URL/Title). Chaves: {obj.keys()}")
except FileNotFoundError:
    log.error(f"Arquivo manifesto não encontrado: {MANIFEST}")
except Exception as e:
    log.error(f"Erro ao ler o manifesto JSONL: {e}")

# Converter para DataFrame para facilidade de manipulação Pandas
df = pd.DataFrame(records)

if df.empty:
    log.error("Nenhum registro válido encontrado no manifesto. Finalizando.")
    dedup = pd.DataFrame()
else:
    df.columns = [c.strip().lower() for c in df.columns]
    df["url_norm"] = df["url"].apply(norm_url)
    
    # Deduplicação baseada na URL normalizada
    dedup = df.drop_duplicates(subset=["url_norm"]).reset_index(drop=True)

print(f"Registros no Manifesto (originais): {len(df)}")
print(f"Registros Únicos (após dedup por URL): {len(dedup)}")
print("\nPrimeiros 3 registros (deduplicados):\n")
print(dedup.head(3).to_markdown(index=False))


Registros no Manifesto (originais): 759
Registros Únicos (após dedup por URL): 184

Primeiros 3 registros (deduplicados):

| title                                                                                        | url                                                                                                         | date       | source_type            | type   |   group | url_norm                                                                                                    |
|:---------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------|:-----------|:-----------------------|:-------|--------:|:------------------------------------------------------------------------------------------------------------|
| Portaria DGP 08/2022 (PC/SP): Tratamento Específico a Travestis e Transexuais nas Delegacias | https://www.stj.jus.br/sites/por

## 5) Funções de extração e limpeza

In [ ]:

import fitz, pdfminer.high_level

EMAIL_RE = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w{2,}\b", re.IGNORECASE)
CPF_RE = re.compile(r"\b\d{3}\.\d{3}\.\d{3}-\d{2}\b")
CNPJ_RE = re.compile(r"\b\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\b")
MULTISPACE = re.compile(r"[ \t]+")
MULTINEWLINE = re.compile(r"\n{3,}")

def clean_text(s: str) -> str:
    if not s: return ""
    s = s.replace("\ufeff", "").replace("-\n","")
    s = MULTISPACE.sub(" ", s)
    s = MULTINEWLINE.sub("\n\n", s)
    s = EMAIL_RE.sub("<EMAIL>", s)
    s = CPF_RE.sub("<CPF>", s)
    s = CNPJ_RE.sub("<CNPJ>", s)
    return s.strip()

def extract_html_main(html_bytes: bytes) -> str:
    if trafilatura is not None:
        try:
            txt = trafilatura.extract(html_bytes, include_comments=False, include_tables=False, favor_recall=True)
            if txt and txt.strip():
                return txt.strip()
        except Exception:
            pass
    try:
        soup = BeautifulSoup(html_bytes, "lxml")
        for tag in soup(["script","style","nav","header","footer","noscript","aside"]):
            tag.decompose()
        return soup.get_text("\n").strip()
    except Exception:
        return ""

def extract_pdf_text(pdf_path: str) -> str:
    if fitz is not None:
        try:
            doc = fitz.open(pdf_path)
            parts = [page.get_text("text") for page in doc]
            out = "\n".join(parts)
            if out.strip():
                return out.strip()
        except Exception as e:
            print(f"fitz extraction failed. ERRO: {e}") 
            
    if pdfminer_extract_text is not None:
        try:
            out = pdfminer_extract_text(pdf_path) or ""
            if out.strip():
                return out.strip()
        except Exception as e:
            print(f"pdfminer extraction failed. ERRO: {e}")
    
    print("Nenhum método de extração funcionou.")
    return ""

print("Text extraction functions defined.")

Text extraction functions defined.


## 6) Baixar e extrair (HTML/PDF)

In [5]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
# Assume-se que 'RAW_DIR', 'TEXT_DIR', 'limiter', 'session', 'sha256', 
# 'extract_pdf_text', 'extract_html_main', 'clean_text', 'MIN_CHARS', 
# e 'dedup' estão definidos em células anteriores.

RAW_DIR = OUTDIR / "data" / "raw"
TEXT_DIR = OUTDIR / "data" / "text"
results = []
download_results = [] # Lista para armazenar resultados da Fase 1

# --- Método 1: Download e Salvamento do Arquivo Bruto ---
def download_raw(row):
    """Realiza a requisição HTTP, salva o arquivo bruto e retorna o status e o caminho."""
    url = row["url"]
    item_type = row.get("type", "").lower()
    url_norm = row["url_norm"] # Usando url_norm já calculada

    # O true pula a verificação de robots.txt
    if not True: #robots.can_fetch(url): 
        return {"url": url, "status": "robots_disallow"}
    
    limiter.acquire()
    
    try:
        resp = session.get(url)
        ct = resp.headers.get("Content-Type", "")
        content = resp.content
        status = resp.status_code
    except Exception as e:
        return {"url": url, "status": f"fetch_error:{e}"}
    
    if status != 200 or not content:
        return {"url": url, "status": f"http_{status}"}
    
    # Determinação do tipo de arquivo, usando 'type' do manifesto como prioridade
    is_pdf = (item_type == "pdf")
    ext = ".pdf" if is_pdf else ".html"
    fname = sha256(url_norm)[:20] + ext
    raw_path = RAW_DIR / fname
    
    try:
        raw_path.write_bytes(content)
    except Exception as e:
        return {"url": url, "status": f"save_raw_error:{e}"}

    print(f"Downloaded and saved: {url} -> {raw_path} -> {item_type}")    
    return {
        "url": url, 
        "status": "download_ok", 
        "raw_path": str(raw_path),
        "is_pdf": is_pdf,
        "content": content # Passa o conteúdo para evitar ler o disco novamente para HTML
    }


# 1. FASE DE DOWNLOAD
print("\n--- FASE 1: DOWNLOAD DOS ARQUIVOS BRUTOS ---")
if 'dedup' in locals() and not dedup.empty:
    dedup_list = dedup.to_dict(orient="records")
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex_fetch:
        futs_fetch = [ex_fetch.submit(download_raw, r) for r in dedup_list]
        for fut_fetch in tqdm(as_completed(futs_fetch), total=len(futs_fetch), desc="Baixando Brutos"):
            download_results.append(fut_fetch.result())

else:
    log.warning("Nenhuma URL para processar após a deduplicação.")



# Saída do bloco original
print(f"\nTotal de resultados: {len(results)}")
print(f"Primeiros 3 resultados: {results[:3]}")

len(results), results[:3]


--- FASE 1: DOWNLOAD DOS ARQUIVOS BRUTOS ---


Baixando Brutos:   1%|          | 1/184 [00:01<04:40,  1.53s/it]

Downloaded and saved: https://www.stj.jus.br/sites/portalp/WebPub/NovoPortal/assets/flip-page/panorama_9_2024_noticias_2023.pdf -> corpus_out_nb/data/raw/f3b1ee82c4c4c7eeae02.pdf -> pdf


Baixando Brutos:   1%|          | 2/184 [00:02<03:20,  1.10s/it]

Downloaded and saved: https://www.ssp.pi.gov.br/wp-content/uploads/2025/06/SSP01_fa3d283ba1.pdf -> corpus_out_nb/data/raw/12ebadbd73215f42a57d.pdf -> pdf
Downloaded and saved: https://social.rs.gov.br/upload/arquivos/202212/06142737-2021-12-procedoperacionalpadrao-pop-idoso-pcdf.pdf -> corpus_out_nb/data/raw/3e74f3867048c2972726.pdf -> pdf


...

Baixando Brutos:  99%|█████████▉| 183/184 [03:05<00:01,  1.56s/it]

Downloaded and saved: https://www.tjrj.jus.br/documents/5736540/6284571/dir-adm.pdf -> corpus_out_nb/data/raw/a9f3b17c91163e8ab545.pdf -> pdf


Baixando Brutos: 100%|██████████| 184/184 [03:07<00:00,  1.02s/it]

Downloaded and saved: https://www.tjam.jus.br/images/ESMAM/Projetos_Esmam/Cartilha_Nossa_dor.pdf -> corpus_out_nb/data/raw/9a743a94ea0ca7a559af.pdf -> pdf

Total de resultados: 0
Primeiros 3 resultados: []


(0, [])

In [38]:
# --- Método 2: Extração e Limpeza dos Dados ---
def process_raw_and_extract(raw_item):
    """Extrai o texto, limpa, verifica o tamanho e salva o arquivo .txt."""
    
    # Retorna itens que falharam na fase anterior
    if raw_item["status"] != "download_ok":
        return raw_item 
        
    url = raw_item["url"]
    raw_path = Path(raw_item["raw_path"])
    is_pdf = raw_item["is_pdf"]
    content = raw_item["content"]
    
    # Início da lógica de extração
    try:
        if is_pdf:
            text = extract_pdf_text(str(raw_path)) 
        else:
            text = extract_html_main(content)
    except Exception as e:
        text = ""
        print(e)
        log.error(f"Extraction error for {url}: {e}")
        
    text = clean_text(text)
    
    # Verifica o tamanho (lógica original)
    if len(text) < MIN_CHARS:
        return {"url": url, "status": "too_short", "chars": len(text)}
        
    # Salvar o texto limpo
    ext_raw = ".pdf" if is_pdf else ".html"
    tname = raw_path.name.replace(ext_raw, ".txt")
    text_path = TEXT_DIR / tname
    
    try:
        text_path.write_text(text, encoding="utf-8")
    except Exception as e:
        return {"url": url, "status": f"save_text_error:{e}"}
        
    # Retorno final de sucesso
    return {
        "url": url, 
        "status": "ok", 
        "text_path": str(text_path), 
        "chars": len(text)
    }

# 2. FASE DE EXTRAÇÃO
print("\n--- FASE 2: EXTRAÇÃO E LIMPEZA DE TEXTO ---")
# Filtra apenas os itens baixados com sucesso para a extração
raw_items_to_process = [r for r in download_results if r["status"] == "download_ok"]
failed_downloads = [r for r in download_results if r["status"] != "download_ok"] # Itens que falharam no download

if raw_items_to_process:
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex_extract:
        futs_extract = [ex_extract.submit(process_raw_and_extract, r) for r in raw_items_to_process]
        for fut_extract in tqdm(as_completed(futs_extract), total=len(futs_extract), desc="Extraindo Texto"):
            results.append(fut_extract.result())
            
    # Adiciona as falhas de download aos resultados finais
    results.extend(failed_downloads)
else:
    log.warning("Nenhum arquivo bruto baixado com sucesso para extração.")

print(f"\nItens processados: {len(raw_items_to_process)}")
print(f"\nItens falhados: {len(failed_downloads)}")


--- FASE 2: EXTRAÇÃO E LIMPEZA DE TEXTO ---
DEBUG: Texto extraído por fitz (31753 chars).


Extraindo Texto:   0%|          | 0/150 [00:00<?, ?it/s]

DEBUG: Texto extraído por fitz (20635 chars).


Extraindo Texto:   2%|▏         | 3/150 [00:00<00:37,  3.91it/s]

DEBUG: Texto extraído por fitz (218071 chars).
DEBUG: Texto extraído por fitz (368422 chars).


...

DEBUG: Texto extraído por fitz (190083 chars).
DEBUG: Texto extraído por fitz (62712 chars).


Extraindo Texto:  97%|█████████▋| 146/150 [00:47<00:02,  1.48it/s]encoding error : input conversion failed due to input error, bytes 0x90 0x64 0xCC 0x5C
encoding error : input conversion failed due to input error, bytes 0x90 0x64 0xCC 0x5C
Extraindo Texto:  99%|█████████▊| 148/150 [01:02<00:09,  4.74s/it]

DEBUG: Texto extraído por fitz (1250181 chars).


Extraindo Texto: 100%|██████████| 150/150 [01:02<00:00,  2.39it/s]


Itens processados: 150

Itens falhados: 34


## 7) Deduplicar por hash e gerar shards JSONL (Mantendo Metadados Originais)

In [44]:

import json

SHARDS_DIR = OUTDIR / "data" / "shards"
SHARDS_DIR.mkdir(parents=True, exist_ok=True)
seen_hash = set()
shard_bytes_limit = int(SHARD_SIZE_MB * 1024 * 1024)
buf = []; size = 0; shard_id = 0

def flush():
    global buf, size, shard_id
    if not buf: return
    out_path = SHARDS_DIR / f"corpus_{shard_id:04d}.jsonl"
    with out_path.open("w", encoding="utf-8") as f:
        for line in buf:
            f.write(line + "\n")
    buf = []; size = 0; shard_id += 1
    log.info("Shard salvo: %s", out_path)

ok_items = [r for r in results if r.get("status") == "ok"]

# Mapear URL_norm de volta para os metadados originais
meta_map = dedup.set_index('url_norm').to_dict('index')

for item in ok_items:
    try:
        text = Path(item["text_path"]).read_text(encoding="utf-8")
    except Exception:
        continue
        
    # Deduplicação por Hash do Conteúdo
    h = sha1(text)
    if h in seen_hash:
        continue
    seen_hash.add(h)
    
    # Resgatar Metadados Originais
    url_norm = norm_url(item["url"])
    metadata = meta_map.get(url_norm, {})
    
    # Criar o objeto JSONL final (estrutura de saída inalterada no formato dos metadados)
    final_record = {
        "text": text
    }
    
    # Filtrar chaves com valor None/NaN para manter a limpeza do JSONL
    final_record = {k: v for k, v in final_record.items() if v is not None and not pd.isna(v)}
    
    line = json.dumps(final_record, ensure_ascii=False)
    buf.append(line); size += len(line.encode("utf-8"))
    
    if size >= shard_bytes_limit:
        flush()
flush()

log.info("Itens OK após extração: %d", len(ok_items))
log.info("Documentos únicos (após deduplicação por hash): %d", len(seen_hash))

len(ok_items), len(seen_hash)


2025-10-30 16:35:42,507 INFO: Shard salvo: corpus_out_nb/data/shards/corpus_0000.jsonl
2025-10-30 16:35:42,508 INFO: Itens OK após extração: 338
2025-10-30 16:35:42,508 INFO: Documentos únicos (após deduplicação por hash): 145


(338, 145)

## 8) Relatório final

In [43]:
# Célula 8 (Sintética): Imprimir Estatística Principal

# A variável 'seen_hash' é assumida como definida no Bloco 7.
# Garante que a variável exista para evitar NameError (fallback de segurança)
if 'seen_hash' not in locals():
    seen_hash = set()

# Calcula a estatística principal
total_documentos_unicos = len(seen_hash)

# Imprime o resultado de forma clara
print("=========================================")
print(f"✅ Documentos Únicos e Válidos Extraídos: {total_documentos_unicos}")
print("=========================================")

# Retorna a variável para manter a consistência do notebook
total_documentos_unicos

✅ Documentos Únicos e Válidos Extraídos: 145


145


## 9) Dicas
- Se um host estiver lento, ajuste: `MAX_WORKERS = 1`, `RATE = 0.5`, `TIMEOUT = 10`.
- Para debug detalhado: `LOG_LEVEL = "DEBUG"`.
- Se precisar **ignorar robots** para diagnóstico rápido, troque `robots.can_fetch(url)` para sempre `True` (apenas para teste).
- Para enriquecer metadados, crie um `index.csv` registrando `url`, `hash`, `path` e `chars` durante o loop.
